# AC温度範囲分析 (A/C ON状態のみ)

このノートブックは、CleaとIsetanMitsukoshiの2つの建物のAC制御データを分析し、A/C ON状態のみのデータを使用して月別の温度統計を計算します。

## 主な機能
- 複数のCSVファイルの読み込みと統合
- A/C ON/OFF状態によるフィルタリング
- A/C Name別の月別温度分析（平均値、標準偏差）
- サンプル数分析とFan Speed頻度分析
- 建物別のExcelファイル生成

## 出力ファイル
- `AC_setvalue_range_analysis_Clea.xlsx`
- `AC_setvalue_range_analysis_IsetanMitsukoshi.xlsx`


In [28]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("ライブラリのインポート完了！")


ライブラリのインポート完了！


In [29]:
# データパス設定（正しいパスに修正）
data_path = "../../Clea/ac-control/"
print(f"データパス: {data_path}")

# 利用可能なファイルの確認
if os.path.exists(data_path):
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"CSVファイル {len(csv_files)}個発見")
    
    # ファイル名サンプル出力
    for i, file in enumerate(csv_files[:5]):
        print(f"  {i+1}. {os.path.basename(file)}")
    
    if len(csv_files) > 5:
        print(f"  ... 他 {len(csv_files)-5}個のファイル")
else:
    print(f"パスが存在しません: {data_path}")
    print("現在の作業ディレクトリ:", os.getcwd())
    print("上位ディレクトリ確認:")
    print("  - ../../Clea/ 存在:", os.path.exists("../../Clea/"))
    print("  - ../../Clea/ac-control/ 存在:", os.path.exists("../../Clea/ac-control/"))


データパス: ../../Clea/ac-control/
CSVファイル 30個発見
  1. ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
  2. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
  3. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
  4. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
  5. ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
  ... 他 25個のファイル


In [30]:
# データローディング関数
def load_csv_data(file_path):
    """CSVファイルをロードする関数"""
    try:
        df = pd.read_csv(file_path)
        print(f"ファイルロード成功: {os.path.basename(file_path)}")
        print(f"  - 行数: {len(df)}")
        print(f"  - 列数: {len(df.columns)}")
        print(f"  - 列名: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"ファイルロードエラー: {e}")
        return None

def list_csv_files(directory):
    """ディレクトリ内のすべてのCSVファイル一覧を返す"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"'{directory}' ディレクトリで {len(csv_files)}個のCSVファイル発見")
    return csv_files

print("データローディング関数定義完了！")


データローディング関数定義完了！


In [31]:
# 使用例（正しいパスに修正）
# 特定ファイルのロード
# df = load_csv_data("../../Clea/ac-control/ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv")

# またはファイル一覧から選択
# csv_files = list_csv_files("../../Clea/ac-control/")
# df = load_csv_data(csv_files[0])  # 最初のファイルをロード

print("データロード準備完了！")
print("上記のコメントを解除して、必要なデータをロードしてください。")
print("正しいパス: ../../Clea/ac-control/")


データロード準備完了！
上記のコメントを解除して、必要なデータをロードしてください。
正しいパス: ../../Clea/ac-control/


## すべてのCSVファイルの統合と月別分析

すべてのAC制御データを統合して月別温度分析を実行します。


In [32]:
# すべてのCSVファイルの統合と月別分析（2つの建物処理）
def load_all_csv_files(directory):
    """ディレクトリ内のすべてのCSVファイルをロードして統合"""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    print(f"総 {len(csv_files)}個のCSVファイル発見")
    
    all_data = []
    
    for i, file_path in enumerate(csv_files):
        print(f"ローディング中... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  エラー: {e}")
            continue
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n統合完了！総 {len(combined_df)}個の行")
        return combined_df
    else:
        print("ロードされたデータがありません。")
        return None

def process_building_data(building_name, data_path):
    """特定の建物のデータを処理"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物データ処理 ===")
    print(f"{'='*60}")
    
    # すべてのデータをロード
    print(f"=== {building_name} CSVファイルロード ===")
    all_data = load_all_csv_files(data_path)
    
    if all_data is None:
        print(f"{building_name} データロード失敗")
        return None
    
    return all_data

# 2つの建物データ処理
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 建物データ処理開始...")
    data = process_building_data(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} データ処理完了: {len(data)}個の行")
    else:
        print(f"{building_name} データ処理失敗")

print(f"\n総 {len(building_data)}個の建物データ処理完了")



Clea 建物データ処理開始...

=== Clea 建物データ処理 ===
=== Clea CSVファイルロード ===
総 30個のCSVファイル発見
ローディング中... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
ローディング中... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
ローディング中... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
ローディング中... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
ローディング中... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
ローディング中... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
ローディング中... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
ローディング中... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
ローディング中... (9

In [ ]:
# データ構造確認と日付列処理
if all_data is not None:
    print("=== データ構造確認 ===")
    print(f"データ形状: {all_data.shape}")
    print(f"列一覧: {list(all_data.columns)}")
    
    # 日付/時間列の検索
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    print(f"時間関連列: {time_columns}")
    
    # 温度関連列の検索
    temp_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['temp', '온도', 'temperature'])]
    print(f"温度関連列: {temp_columns}")
    
    # サンプルデータ確認
    print(f"\nサンプルデータ（最初の3行）:")
    print(all_data.head(3))
    
    # 各列のデータ型確認
    print(f"\nデータ型:")
    print(all_data.dtypes)


=== 建物データから分析用データを設定 ===
Clea建物データを使用: 3400320個の行
=== データ構造確認 ===
データ形状: (3400320, 14)
列一覧: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'A/C ON/OFF', 'A/C Mode', 'A/C Fan Speed', 'Naive Energy Level', 'Airux Energy Level', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.', 'source_file']
時間関連列: ['Datetime']
温度関連列: ['Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature', 'Outdoor Room Temp.', 'Outdoor Set Temp.', 'Room Set Temp.']

サンプルデータ（最初の3行）:
  A/C Name                          Datetime  Outdoor Temp.  Indoor Temp.  \
0     G-24  2025-04-30 23:55:00.084567+09:00            6.0          16.0   
1     G-21  2025-04-30 23:55:00.084567+09:00            6.0          20.0   
2     G-23  2025-04-30 23:55:00.084567+09:00            6.0          16.0   

   A/C Set Temperature A/C ON/OFF A/C Mode A/C Fan Speed  Naive Energy Level  \
0                 26.0        OFF     COOL           NaN                   0   
1                 26.0        OFF

In [45]:
# 日付列処理と月別分析（A/C ON状態のみフィルタリング）
def process_date_column(df, date_column):
    """日付列を処理して月情報を抽出"""
    if date_column not in df.columns:
        print(f"日付列 '{date_column}' が存在しません。")
        return df
    
    # 日付列をdatetimeに変換
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    
    # 月情報抽出
    df['year_month'] = df[date_column].dt.to_period('M')
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    
    return df

def analyze_monthly_temperature(df, indoor_temp_col, ac_set_temp_col):
    """月別温度分析（A/C ON状態のみ）"""
    if indoor_temp_col not in df.columns or ac_set_temp_col not in df.columns:
        print(f"必要な温度列が存在しません。")
        print(f"Indoor Temp: {indoor_temp_col in df.columns}")
        print(f"A/C Set Temperature: {ac_set_temp_col in df.columns}")
        return None
    
    # 月別グループ化と統計計算
    monthly_stats = df.groupby('year_month').agg({
        indoor_temp_col: ['mean', 'std', 'count'],
        ac_set_temp_col: ['mean', 'std', 'count']
    }).round(2)
    
    # 列名整理
    monthly_stats.columns = [
        'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
        'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
    ]
    
    # インデックスを文字列に変換
    monthly_stats.index = monthly_stats.index.astype(str)
    
    return monthly_stats

# 日付列処理とA/C ON状態フィルタリング
if all_data is not None:
    time_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    
    if time_columns:
        date_column = time_columns[0]
        print(f"日付列 '{date_column}' 使用")
        all_data = process_date_column(all_data, date_column)
        
        # A/C ON/OFF列検索とフィルタリング
        ac_onoff_columns = [col for col in all_data.columns if any(keyword in col.lower() for keyword in ["A/C ON/OFF", 'on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
        print(f"A/C ON/OFF関連列: {ac_onoff_columns}")
        
        if ac_onoff_columns:
            ac_onoff_col = ac_onoff_columns[0]
            print(f"A/C ON/OFF列 '{ac_onoff_col}' 使用")
            
            # フィルタリング前データ数
            total_rows = len(all_data)
            print(f"全体データ: {total_rows}個の行")
            
            # A/C ON状態のみフィルタリング
            ac_on_data = all_data[all_data[ac_onoff_col] == 'ON'].copy()
            ac_on_rows = len(ac_on_data)
            print(f"A/C ON状態データ: {ac_on_rows}個の行 ({ac_on_rows/total_rows*100:.1f}%)")
            
            if ac_on_rows > 0:
                # 月別分析（A/C ON状態のみ）
                indoor_temp_col = 'Indoor Temp.'
                ac_set_temp_col = 'A/C Set Temperature'
                
                monthly_analysis = analyze_monthly_temperature(ac_on_data, indoor_temp_col, ac_set_temp_col)
                
                if monthly_analysis is not None:
                    print("\n=== A/C ON状態月別温度分析結果 ===")
                    print(monthly_analysis)
                    
                    # フィルタリングされたデータをall_dataに更新
                    all_data = ac_on_data
                else:
                    print("A/C ON状態データで月別分析を実行できません。")
            else:
                print("A/C ON状態データがありません。")
        else:
            print("A/C ON/OFF列が見つかりません。全体データで分析を進行します。")
            
            # 月別分析（全体データ）
            indoor_temp_col = 'Indoor Temp.'
            ac_set_temp_col = 'A/C Set Temperature'
            
            monthly_analysis = analyze_monthly_temperature(all_data, indoor_temp_col, ac_set_temp_col)
            
            if monthly_analysis is not None:
                print("\n=== 月別温度分析結果 ===")
                print(monthly_analysis)
            else:
                print("月別分析を実行できません。")
    else:
        print("日付/時間列が見つかりません。")


日付列 'Datetime' 使用
A/C ON/OFF関連列: ['A/C ON/OFF']
A/C ON/OFF列 'A/C ON/OFF' 使用
全体データ: 983682個の行
A/C ON状態データ: 983682個の行 (100.0%)

=== A/C ON状態月別温度分析結果 ===
            Indoor_Temp_Mean  Indoor_Temp_Std  Indoor_Temp_Count  \
year_month                                                         
2024-07                27.43             1.89              84927   
2024-08                26.83             2.15              87083   
2024-09                25.69             2.00              83463   
2024-10                25.47             2.07              52638   
2024-11                23.88             2.80              41735   
2024-12                25.02             3.34              65915   
2025-01                24.56             3.77              55788   
2025-02                24.34             3.69              51607   
2025-03                25.77             2.80              56233   
2025-04                25.24             2.35              36186   
2025-05                25.44     

In [46]:
# 既存Excelファイル確認とA/C Name別分析（A/C ON状態のみ）
def check_existing_excel(filename):
    """既存Excelファイルのシート構造確認"""
    try:
        if os.path.exists(filename):
            # 既存ファイル読み込み
            excel_file = pd.ExcelFile(filename)
            print(f"既存Excelファイル発見: {filename}")
            print(f"既存シート一覧: {excel_file.sheet_names}")
            
            # 各シートの構造確認
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(filename, sheet_name=sheet_name)
                print(f"\n=== {sheet_name} シート ===")
                print(f"行数: {len(df)}, 列数: {len(df.columns)}")
                print(f"列: {list(df.columns)}")
                if len(df) > 0:
                    print(f"サンプルデータ:")
                    print(df.head(3))
            return excel_file
        else:
            print(f"既存Excelファイルがありません: {filename}")
            return None
    except Exception as e:
        print(f"Excelファイル確認エラー: {e}")
        return None

def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name別で月別温度分析（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns:
        print("A/C Name列が存在しません。")
        return None
    
    # A/C Name別でグループ化
    ac_names = df['A/C Name'].unique()
    print(f"発見されたA/C Name: {ac_names}")
    
    results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 分析中 ===")
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            continue
            
        # 月別グループ化と統計計算
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 列名整理
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # インデックスを文字列に変換
        monthly_stats.index = monthly_stats.index.astype(str)
        
        results[ac_name] = monthly_stats
        print(f"  - 分析完了: {len(monthly_stats)}個の月")
    
    return results

# 既存Excelファイル確認
filename = "AC_setvalue_range_analysis.xlsx"
existing_excel = check_existing_excel(filename)


既存Excelファイル発見: AC_setvalue_range_analysis.xlsx
既存シート一覧: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度']

=== Indoortemp平均 シート ===
行数: 12, 列数: 27
列: ['Unnamed: 0', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']
サンプルデータ:
  Unnamed: 0   G-22   G-21   G-24   G-23  D-1南1  D-3南2  D-6北1  D-5南1   A-26  \
0         1月  18.18  18.44  19.78    NaN  25.83  26.73  27.40  26.79  20.75   
1         2月  19.79  19.44  22.02    NaN  25.99  26.28  28.11  27.56  21.15   
2         3月  23.18  23.27  25.65  22.19  26.96  27.17  23.79  24.34  23.58   

   ...  D-4北2  E-15北3  D-7南2  D-8北2  E-16北4   A-25  E-12南4  D-2北1   F-19  \
0  ...  27.39   25.78  27.81  27.62   24.13  17.42   25.67   27.4  24.80   
1  ...    NaN   25.02    NaN    NaN   24.28  16.70   25.56    NaN  25.35   
2  ...  25.2

In [ ]:
# A/C Name別分析実行とExcel更新（A/C ON状態のみ、既存フォーマットに合わせて）
if all_data is not None and 'A/C Name' in all_data.columns:
    print("\n=== A/C Name別月別分析実行（A/C ON状態のみ） ===")
    
    indoor_temp_col = 'Indoor Temp.'
    ac_set_temp_col = 'A/C Set Temperature'
    
    # A/C Name別分析
    ac_analysis_results = analyze_by_ac_name(all_data, indoor_temp_col, ac_set_temp_col)
    
    if ac_analysis_results:
        print(f"\n総 {len(ac_analysis_results)}個のA/C Nameに対する分析完了")
        
        # 各A/C Nameの分析結果プレビュー
        for ac_name, monthly_data in ac_analysis_results.items():
            print(f"\n--- {ac_name} 分析結果 ---")
            print(f"分析期間: {len(monthly_data)}個の月")
            print(f"月別平均 Indoor Temp: {monthly_data['Indoor_Temp_Mean'].mean():.2f}°C")
            print(f"月別平均 A/C Set Temp: {monthly_data['AC_Set_Temp_Mean'].mean():.2f}°C")
        
        # A/C ON状態サンプル数計算
        def calculate_ac_on_sample_counts(df):
            """A/C ON状態の月別サンプル数計算"""
            if 'A/C Name' not in df.columns:
                return None
            
            # A/C Name別月別サンプル数計算
            sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
            
            # 月別インデックス生成（1月, 2月, ..., 12月）
            months = [f"{i}月" for i in range(1, 13)]
            
            # A/C Name別データを月別に整理
            monthly_sample_data = {}
            
            for ac_name in df['A/C Name'].unique():
                monthly_values = {}
                
                # 該当A/C Nameの月別サンプル数
                ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
                
                for _, row in ac_data.iterrows():
                    # periodを月に変換（例: 2024-07 -> 7月）
                    month_period = row['year_month']
                    month_num = int(str(month_period).split('-')[1])
                    month_name = f"{month_num}月"
                    monthly_values[month_name] = row['sample_count']
                
                monthly_sample_data[ac_name] = monthly_values
            
            # DataFrame生成（月別が行、A/C Nameが列）
            df_samples = pd.DataFrame(monthly_sample_data, index=months)
            
            # 最初の列を月情報として追加
            df_samples.insert(0, 'Unnamed: 0', months)
            
            return df_samples
        
        # A/C Fan Speed頻度計算
        def calculate_fan_speed_frequency(df):
            """A/C Fan Speedの月別頻度計算（A/C ON状態のみ）"""
            if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
                print("A/C NameまたはA/C Fan Speed列がありません。")
                return None
            
            # A/C ON/OFF列がある場合はON状態のみフィルタリング
            ac_onoff_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
            
            if ac_onoff_columns:
                ac_onoff_col = ac_onoff_columns[0]
                print(f"Fan Speed頻度計算: A/C ON/OFF列 '{ac_onoff_col}' 使用")
                
                # A/C ON状態のみフィルタリング
                ac_on_data = df[df[ac_onoff_col] == 'ON'].copy()
                print(f"Fan Speed頻度計算用データ: {len(ac_on_data)}個の行（A/C ON状態のみ）")
                
                if len(ac_on_data) == 0:
                    print("A/C ON状態データがないためFan Speed頻度を計算できません。")
                    return None
                
                # フィルタリングされたデータ使用
                df = ac_on_data
            else:
                print("A/C ON/OFF列が見つからないため全体データでFan Speed頻度を計算します。")
            
            # 月別インデックス生成（1月, 2月, ..., 12月）
            months = [f"{i}月" for i in range(1, 13)]
            
            # Fan Speed固有値確認
            fan_speeds = df['A/C Fan Speed'].unique()
            print(f"発見されたFan Speed値: {fan_speeds}")
            
            # Fan Speed別月別頻度計算
            fan_speed_frequency_data = []
            
            for month_name in months:
                month_num = int(month_name.replace('月', ''))
                
                for fan_speed in fan_speeds:
                    # 該当月の該当Fan Speed頻度計算
                    month_data = df[df['month'] == month_num]
                    fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
                    
                    fan_speed_frequency_data.append({
                        'Unnamed: 0': month_name,
                        'Unnamed: 1': fan_speed,
                        'frequency': fan_speed_count
                    })
            
            # DataFrame生成
            df_frequency = pd.DataFrame(fan_speed_frequency_data)
            
            # A/C Name別で列追加
            for ac_name in df['A/C Name'].unique():
                ac_frequency_values = []
                
                for month_name in months:
                    month_num = int(month_name.replace('月', ''))
                    
                    for fan_speed in fan_speeds:
                        # 該当A/C Nameの該当月、該当Fan Speed頻度
                        month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                        fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                        ac_frequency_values.append(fan_speed_count)
                
                df_frequency[ac_name] = ac_frequency_values
            
            print(f"Fan Speed頻度計算完了: {len(df_frequency)}個の行")
            return df_frequency
        
        # A/C ON状態サンプル数計算
        sample_counts_df = calculate_ac_on_sample_counts(all_data)
        
        # A/C Fan Speed頻度計算
        fan_speed_frequency_df = calculate_fan_speed_frequency(all_data)
        
        # Excelファイル更新（既存フォーマット維持）
        def update_excel_with_ac_data(excel_file, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename):
            """既存ExcelファイルにA/C Name別データを既存フォーマットに合わせて入力（A/C ON状態のみ）"""
            try:
                # 既存ファイルがある場合は読み込み
                if excel_file is not None:
                    existing_sheets = {}
                    for sheet_name in excel_file.sheet_names:
                        existing_sheets[sheet_name] = pd.read_excel(filename, sheet_name=sheet_name)
                else:
                    existing_sheets = {}
                
                # 新しいExcelファイル生成
                with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                    # 既存シート維持しながら更新
                    for sheet_name, sheet_data in existing_sheets.items():
                        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
                    
                    # A/C Name別データを既存フォーマットに合わせて各シートに入力
                    if ac_analysis_results:
                        # 月別データを既存フォーマットに変換する関数
                        def create_monthly_dataframe(ac_analysis_results, data_type):
                            """A/C Name別分析結果を既存フォーマットのDataFrameに変換（A/C ON状態のみ）"""
                            # 月別インデックス生成（1月, 2月, ..., 12月）
                            months = [f"{i}月" for i in range(1, 13)]
                            
                            # A/C Name別データを月別に整理
                            monthly_data = {}
                            
                            for ac_name, monthly_stats in ac_analysis_results.items():
                                monthly_values = {}
                                
                                for month_period, row in monthly_stats.iterrows():
                                    # periodを月に変換（例: 2024-07 -> 7月）
                                    month_num = int(month_period.split('-')[1])
                                    month_name = f"{month_num}月"
                                    
                                    if data_type == 'indoor_mean':
                                        monthly_values[month_name] = row['Indoor_Temp_Mean']
                                    elif data_type == 'indoor_std':
                                        monthly_values[month_name] = row['Indoor_Temp_Std']
                                    elif data_type == 'ac_set_mean':
                                        monthly_values[month_name] = row['AC_Set_Temp_Mean']
                                    elif data_type == 'ac_set_std':
                                        monthly_values[month_name] = row['AC_Set_Temp_Std']
                                
                                monthly_data[ac_name] = monthly_values
                            
                            # DataFrame生成（月別が行、A/C Nameが列）
                            df = pd.DataFrame(monthly_data, index=months)
                            
                            # 最初の列を月情報として追加
                            df.insert(0, 'Unnamed: 0', months)
                            
                            return df
                        
                        # 1. Indoortemp平均シート（Indoor Temp月別平均 - A/C ON状態のみ）
                        indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                        indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                        
                        # 2. 設定温度_平均値シート（A/C Set Temperature月別平均 - A/C ON状態のみ）
                        ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                        ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                        
                        # 3. Indoortemp標準偏差シート（Indoor Temp月別標準偏差 - A/C ON状態のみ）
                        indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                        indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                        
                        # 4. 設定温度_標準偏差シート（A/C Set Temperature月別標準偏差 - A/C ON状態のみ）
                        ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                        ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
                    
                    # 5. 室内機別_サンプル数シート（A/C ON状態サンプル数）
                    if sample_counts_df is not None:
                        sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                        print("室内機別_サンプル数シートにA/C ON状態サンプル数入力完了")
                    
                    # 6. FanSpeed頻度シート（A/C Fan Speed頻度）
                    if fan_speed_frequency_df is not None:
                        fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                        print("FanSpeed頻度シートにA/C Fan Speed頻度入力完了")
                
                print(f"Excelファイル更新完了: {filename}")
                return True
                
            except Exception as e:
                print(f"Excelファイル更新エラー: {e}")
                return False
        
        # Excelファイル更新実行
        print("\n=== Excelファイル更新実行（A/C ON状態のみ） ===")
        success = update_excel_with_ac_data(existing_excel, ac_analysis_results, sample_counts_df, fan_speed_frequency_df, filename)
        
        if success:
            print(f"\n=== 更新されたシート一覧（A/C ON状態のみ） ===")
            print("1. Indoortemp平均 - Indoor Temp月別平均（A/C ON状態のみ）")
            print("2. 設定温度_平均値 - A/C Set Temperature月別平均（A/C ON状態のみ）") 
            print("3. Indoortemp標準偏差 - Indoor Temp月別標準偏差（A/C ON状態のみ）")
            print("4. 設定温度_標準偏差 - A/C Set Temperature月別標準偏差（A/C ON状態のみ）")
            print("5. 室内機別_サンプル数 - A/C ON状態月別サンプル数")
            print("6. FanSpeed頻度 - A/C Fan Speed月別頻度")
            
            # 更新されたファイル確認
            updated_excel = pd.ExcelFile(filename)
            print(f"\n最終シート一覧: {updated_excel.sheet_names}")
        else:
            print("Excelファイル更新に失敗しました。")
    else:
        print("A/C Name別分析を実行できません。")
else:
    print("A/C Name列がないかデータがありません。")
    if all_data is not None:
        print(f"利用可能な列: {list(all_data.columns)}")


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 45)

In [ ]:
# MASTER_Clea.xlsx의 制御マスタ 시트 구조 확인
import pandas as pd

print("=== MASTER_Clea.xlsx 制御マスタ 시트 구조 확인 ===")
try:
    master_file = "MASTER_Clea.xlsx"
    master_excel = pd.ExcelFile(master_file)
    
    print(f"전체 시트 목록: {master_excel.sheet_names}")
    
    # 制御マスタ 시트가 있는지 확인
    if "制御マスタ" in master_excel.sheet_names:
        print(f"\n✅ 制御マスタ 시트 발견!")
        
        # 制御マスタ 시트 읽기
        control_master_df = pd.read_excel(master_file, sheet_name="制御マスタ")
        
        print(f"\n制御マスタ 시트 구조:")
        print(f"  행수: {len(control_master_df)}")
        print(f"  열수: {len(control_master_df.columns)}")
        print(f"  열명: {list(control_master_df.columns)}")
        
        print(f"\n制御マスタ 시트 샘플 데이터:")
        print(control_master_df.head(10))
        
        print(f"\n制御マスタ 시트 데이터 타입:")
        print(control_master_df.dtypes)
        
        # A/C Name 관련 열이 있는지 확인
        ac_name_columns = [col for col in control_master_df.columns if 'name' in col.lower() or 'ac' in col.lower() or 'name' in col.lower()]
        print(f"\nA/C Name 관련 열: {ac_name_columns}")
        
    else:
        print(f"\n❌ 制御マスタ 시트를 찾을 수 없습니다.")
        print(f"사용 가능한 시트: {master_excel.sheet_names}")
        
        # 비슷한 이름의 시트가 있는지 확인
        similar_sheets = [sheet for sheet in master_excel.sheet_names if '制御' in sheet or 'マスタ' in sheet or 'master' in sheet.lower()]
        if similar_sheets:
            print(f"비슷한 이름의 시트: {similar_sheets}")
            
except Exception as e:
    print(f"파일 읽기 오류: {e}")


In [ ]:
# 既存Excelファイルのシート構造詳細確認（FanSpeed頻度シート含む）
filename = "AC_setvalue_range_analysis.xlsx"

if os.path.exists(filename):
    print(f"=== {filename} ファイル構造分析 ===")
    
    # Excelファイル読み込み
    excel_file = pd.ExcelFile(filename)
    print(f"シート一覧: {excel_file.sheet_names}")
    
    # FanSpeed頻度シートがあるか確認
    fanspeed_sheet_name = None
    for sheet_name in excel_file.sheet_names:
        if 'FanSpeed' in sheet_name or '頻度' in sheet_name:
            fanspeed_sheet_name = sheet_name
            break
    
    if fanspeed_sheet_name:
        print(f"\n{'='*60}")
        print(f"発見されたFanSpeed頻度シート: {fanspeed_sheet_name}")
        print(f"{'='*60}")
        
        df = pd.read_excel(filename, sheet_name=fanspeed_sheet_name)
        print(f"行数: {len(df)}, 列数: {len(df.columns)}")
        print(f"インデックスタイプ: {type(df.index)}")
        print(f"インデックス値: {list(df.index)}")
        print(f"列名: {list(df.columns)}")
        
        if len(df) > 0:
            print(f"\n全体データ:")
            print(df)
            
            # 各列のデータ型確認
            print(f"\nデータ型:")
            print(df.dtypes)
            
            # NaN値確認
            print(f"\nNaN値個数:")
            print(df.isnull().sum())
            
            # 最初の列の固有値確認（月情報）
            if 'Unnamed: 0' in df.columns:
                print(f"\n最初の列（月）固有値:")
                print(df['Unnamed: 0'].unique())
            
            # 2番目の列の固有値確認（Fan Speed情報）
            if 'Unnamed: 1' in df.columns:
                print(f"\n2番目の列（Fan Speed）固有値:")
                print(df['Unnamed: 1'].unique())
        else:
            print("データが空です。")
    else:
        print("FanSpeed頻度関連シートが見つかりません。")
    
    # 他のシートも簡単に確認
    for sheet_name in excel_file.sheet_names:
        if sheet_name != fanspeed_sheet_name:
            print(f"\n{'='*30}")
            print(f"シート名: {sheet_name}")
            print(f"{'='*30}")
            
            df = pd.read_excel(filename, sheet_name=sheet_name)
            print(f"行数: {len(df)}, 列数: {len(df.columns)}")
            print(f"列名: {list(df.columns)}")
            
            if len(df) > 0:
                print(f"\nサンプルデータ:")
                print(df.head(3))
else:
    print(f"ファイルが存在しません: {filename}")


=== AC_setvalue_range_analysis.xlsx ファイル構造分析 ===
シート一覧: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度']

発見されたFanSpeed頻度シート: FanSpeed頻度
行数: 60, 列数: 29
インデックスタイプ: <class 'pandas.core.indexes.range.RangeIndex'>
インデックス値: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
列名: ['Unnamed: 0', 'Unnamed: 1', 'frequency', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

全体データ:
   Unnamed: 0 Unnamed: 1  frequency  G-22  G-21  G-24  G-23  D-1南1  D-3南2  \
0          1月     Medium      19630  1286  1626    50     0   6642   6642   
1          1月        Low      35902     0     0     0     0   2206   220

In [49]:
# 2つの建物データ処理と分析
def process_building_analysis(building_name, data_path):
    """特定の建物のデータをロードして分析"""
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物データ処理 ===")
    print(f"{'='*60}")
    
    # データロード
    csv_files = glob.glob(os.path.join(data_path, "*.csv"))
    print(f"{building_name}: 総 {len(csv_files)}個のCSVファイル発見")
    
    all_data = []
    for i, file_path in enumerate(csv_files):
        print(f"ローディング中... ({i+1}/{len(csv_files)}) {os.path.basename(file_path)}")
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"  エラー: {e}")
            continue
    
    if not all_data:
        print(f"{building_name} データロード失敗")
        return None
    
    # データ統合
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"{building_name} 統合完了: {len(combined_df)}個の行")
    
    # 日付列処理
    time_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'datetime'])]
    if time_columns:
        date_column = time_columns[0]
        print(f"{building_name} 日付列 '{date_column}' 使用")
        combined_df[date_column] = pd.to_datetime(combined_df[date_column], errors='coerce')
        combined_df['year_month'] = combined_df[date_column].dt.to_period('M')
        combined_df['year'] = combined_df[date_column].dt.year
        combined_df['month'] = combined_df[date_column].dt.month
    
    # A/C ON/OFFフィルタリング
    ac_onoff_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['on/off', 'onoff', 'on_off', 'ac_on', 'ac_off'])]
    if ac_onoff_columns:
        ac_onoff_col = ac_onoff_columns[0]
        print(f"{building_name} A/C ON/OFF列 '{ac_onoff_col}' 使用")
        
        total_rows = len(combined_df)
        ac_on_data = combined_df[combined_df[ac_onoff_col] == 'ON'].copy()
        ac_on_rows = len(ac_on_data)
        print(f"{building_name} A/C ON状態データ: {ac_on_rows}個の行 ({ac_on_rows/total_rows*100:.1f}%)")
        
        if ac_on_rows > 0:
            return ac_on_data
        else:
            print(f"{building_name} A/C ON状態データがありません。")
            return None
    else:
        print(f"{building_name} A/C ON/OFF列が見つかりません。")
        return combined_df

# 2つの建物データ処理
buildings = {
    'Clea': '../../Clea/ac-control/',
    'IsetanMitsukoshi': '../../IsetanMitsukoshi/ac-control/'
}

building_data = {}

for building_name, data_path in buildings.items():
    print(f"\n{building_name} 建物データ処理開始...")
    data = process_building_analysis(building_name, data_path)
    if data is not None:
        building_data[building_name] = data
        print(f"{building_name} データ処理完了: {len(data)}個の行")
    else:
        print(f"{building_name} データ処理失敗")

print(f"\n総 {len(building_data)}個の建物データ処理完了")



Clea 建物データ処理開始...

=== Clea 建物データ処理 ===
Clea: 総 30個のCSVファイル発見
ローディング中... (1/30) ac-control-71,70,72,73-logs-2025-04-01-2025-04-30.csv
ローディング中... (2/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv
ローディング中... (3/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-07-01-2024-07-31.csv
ローディング中... (4/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-02-01-2025-02-28.csv
ローディング中... (5/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-08-01-2025-08-31.csv
ローディング中... (6/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2024-12-01-2024-12-31.csv
ローディング中... (7/30) ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-01-01-2025-01-31.csv
ローディング中... (8/30) ac-control-71,70,72,73-logs-2024-11-01-2024-11-30.csv
ローディング中... (9/30) ac-control-50

In [61]:
# 各建物別分析とExcelファイル生成
def analyze_by_ac_name(df, indoor_temp_col, ac_set_temp_col):
    """A/C Name別月別温度分析（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns:
        print("A/C Name列がありません。")
        return None
    
    ac_names = df['A/C Name'].unique()
    print(f"総 {len(ac_names)}個のA/C Name発見")
    
    ac_analysis_results = {}
    
    for ac_name in ac_names:
        print(f"\n=== {ac_name} 分析中 ===")
        
        # 該当A/C Nameのデータのみフィルタリング
        ac_data = df[df['A/C Name'] == ac_name].copy()
        
        if len(ac_data) == 0:
            print(f"  {ac_name} データがありません。")
            continue
        
        # 月別グループ化と統計計算
        monthly_stats = ac_data.groupby('year_month').agg({
            indoor_temp_col: ['mean', 'std', 'count'],
            ac_set_temp_col: ['mean', 'std', 'count']
        }).round(2)
        
        # 列名整理
        monthly_stats.columns = [
            'Indoor_Temp_Mean', 'Indoor_Temp_Std', 'Indoor_Temp_Count',
            'AC_Set_Temp_Mean', 'AC_Set_Temp_Std', 'AC_Set_Temp_Count'
        ]
        
        # インデックスを文字列に変換
        monthly_stats.index = monthly_stats.index.astype(str)
        
        ac_analysis_results[ac_name] = monthly_stats
        print(f"  分析完了: {len(monthly_stats)}個の月")
    
    return ac_analysis_results

def calculate_ac_on_sample_counts(df):
    """A/C ON状態の月別サンプル数計算"""
    if 'A/C Name' not in df.columns:
        return None
    
    # A/C Name別月別サンプル数計算
    sample_counts = df.groupby(['A/C Name', 'year_month']).size().reset_index(name='sample_count')
    
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name別データを月別に整理
    monthly_sample_data = {}
    
    for ac_name in df['A/C Name'].unique():
        monthly_values = {}
        
        # 該当A/C Nameの月別サンプル数
        ac_data = sample_counts[sample_counts['A/C Name'] == ac_name]
        
        for _, row in ac_data.iterrows():
            # periodを月に変換（例: 2024-07 -> 7月）
            month_period = row['year_month']
            month_num = int(str(month_period).split('-')[1])
            month_name = f"{month_num}月"
            monthly_values[month_name] = row['sample_count']
        
        monthly_sample_data[ac_name] = monthly_values
    
    # DataFrame生成（月別が行、A/C Nameが列）
    df_samples = pd.DataFrame(monthly_sample_data, index=months)
    
    # 最初の列を月情報として追加
    df_samples.insert(0, 'Unnamed: 0', months)
    
    return df_samples

def calculate_fan_speed_frequency(df):
    """A/C Fan Speedの月別頻度計算（A/C ON状態のみ）"""
    if 'A/C Name' not in df.columns or 'A/C Fan Speed' not in df.columns:
        print("A/C NameまたはA/C Fan Speed列がありません。")
        return None
    
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # Fan Speed固有値確認
    fan_speeds = df['A/C Fan Speed'].unique()
    print(f"発見されたFan Speed値: {fan_speeds}")
    
    # Fan Speed別月別頻度計算
    fan_speed_frequency_data = []
    
    for month_name in months:
        month_num = int(month_name.replace('月', ''))
        
        for fan_speed in fan_speeds:
            # 該当月の該当Fan Speed頻度計算
            month_data = df[df['month'] == month_num]
            fan_speed_count = len(month_data[month_data['A/C Fan Speed'] == fan_speed])
            
            fan_speed_frequency_data.append({
                'Unnamed: 0': month_name,
                'Unnamed: 1': fan_speed,
                'frequency': fan_speed_count
            })
    
    # DataFrame生成
    df_frequency = pd.DataFrame(fan_speed_frequency_data)
    
    # A/C Name別で列追加
    for ac_name in df['A/C Name'].unique():
        ac_frequency_values = []
        
        for month_name in months:
            month_num = int(month_name.replace('月', ''))
            
            for fan_speed in fan_speeds:
                # 該当A/C Nameの該当月、該当Fan Speed頻度
                month_ac_data = df[(df['month'] == month_num) & (df['A/C Name'] == ac_name)]
                fan_speed_count = len(month_ac_data[month_ac_data['A/C Fan Speed'] == fan_speed])
                ac_frequency_values.append(fan_speed_count)
        
        df_frequency[ac_name] = ac_frequency_values
    
    print(f"Fan Speed頻度計算完了: {len(df_frequency)}個の行")
    return df_frequency

def create_monthly_dataframe(ac_analysis_results, data_type):
    """A/C Name別分析結果を既存フォーマットのDataFrameに変換（A/C ON状態のみ）"""
    # 月別インデックス生成（1月, 2月, ..., 12月）
    months = [f"{i}月" for i in range(1, 13)]
    
    # A/C Name別データを月別に整理
    monthly_data = {}
    
    for ac_name, monthly_stats in ac_analysis_results.items():
        monthly_values = {}
        
        for month_period, row in monthly_stats.iterrows():
            # periodを月に変換（例: 2024-07 -> 7月）
            month_num = int(month_period.split('-')[1])
            month_name = f"{month_num}月"
            
            if data_type == 'indoor_mean':
                monthly_values[month_name] = row['Indoor_Temp_Mean']
            elif data_type == 'indoor_std':
                monthly_values[month_name] = row['Indoor_Temp_Std']
            elif data_type == 'ac_set_mean':
                monthly_values[month_name] = row['AC_Set_Temp_Mean']
            elif data_type == 'ac_set_std':
                monthly_values[month_name] = row['AC_Set_Temp_Std']
        
        monthly_data[ac_name] = monthly_values
    
    # DataFrame生成（月別が行、A/C Nameが列）
    df = pd.DataFrame(monthly_data, index=months)
    
    # 最初の列を月情報として追加
    df.insert(0, 'Unnamed: 0', months)
    
    return df

def update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df):
    """建物別Excelファイル生成"""
    filename = f"AC_setvalue_range_analysis_{building_name}.xlsx"
    
    try:
        # 新しいExcelファイル生成
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # A/C Name別データを既存フォーマットに合わせて各シートに入力
            if ac_analysis_results:
                # 1. Indoortemp平均シート（Indoor Temp月別平均 - A/C ON状態のみ）
                indoor_avg_df = create_monthly_dataframe(ac_analysis_results, 'indoor_mean')
                indoor_avg_df.to_excel(writer, sheet_name='Indoortemp平均', index=False)
                
                # 2. 設定温度_平均値シート（A/C Set Temperature月別平均 - A/C ON状態のみ）
                ac_set_avg_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_mean')
                ac_set_avg_df.to_excel(writer, sheet_name='設定温度_平均値', index=False)
                
                # 3. Indoortemp標準偏差シート（Indoor Temp月別標準偏差 - A/C ON状態のみ）
                indoor_std_df = create_monthly_dataframe(ac_analysis_results, 'indoor_std')
                indoor_std_df.to_excel(writer, sheet_name='Indoortemp標準偏差', index=False)
                
                # 4. 設定温度_標準偏差シート（A/C Set Temperature月別標準偏差 - A/C ON状態のみ）
                ac_set_std_df = create_monthly_dataframe(ac_analysis_results, 'ac_set_std')
                ac_set_std_df.to_excel(writer, sheet_name='設定温度_標準偏差', index=False)
            
            # 5. 室内機別_サンプル数シート（A/C ON状態サンプル数）
            if sample_counts_df is not None:
                sample_counts_df.to_excel(writer, sheet_name='室内機別_サンプル数', index=False)
                print(f"{building_name}: 室内機別_サンプル数シートにA/C ON状態サンプル数入力完了")
            
            # 6. FanSpeed頻度シート（A/C Fan Speed頻度）
            if fan_speed_frequency_df is not None:
                fan_speed_frequency_df.to_excel(writer, sheet_name='FanSpeed頻度', index=False)
                print(f"{building_name}: FanSpeed頻度シートにA/C Fan Speed頻度入力完了")
        
        print(f"{building_name}: Excelファイル生成完了 - {filename}")
        return True
        
    except Exception as e:
        print(f"{building_name}: Excelファイル生成エラー - {e}")
        return False

# 各建物別分析とExcelファイル生成
for building_name, data in building_data.items():
    print(f"\n{'='*60}")
    print(f"=== {building_name} 建物分析とExcelファイル生成 ===")
    print(f"{'='*60}")
    
    if 'A/C Name' in data.columns:
        indoor_temp_col = 'Indoor Temp.'
        ac_set_temp_col = 'A/C Set Temperature'
        
        # A/C Name別分析
        ac_analysis_results = analyze_by_ac_name(data, indoor_temp_col, ac_set_temp_col)
        
        if ac_analysis_results:
            print(f"{building_name}: 総 {len(ac_analysis_results)}個のA/C Nameに対する分析完了")
            
            # A/C ON状態サンプル数計算
            sample_counts_df = calculate_ac_on_sample_counts(data)
            
            # A/C Fan Speed頻度計算
            fan_speed_frequency_df = calculate_fan_speed_frequency(data)
            
            # Excelファイル生成
            success = update_excel_with_building_data(building_name, ac_analysis_results, sample_counts_df, fan_speed_frequency_df)
            
            if success:
                print(f"\n=== {building_name} 更新されたシート一覧（A/C ON状態のみ） ===")
                print("1. Indoortemp平均 - Indoor Temp月別平均（A/C ON状態のみ）")
                print("2. 設定温度_平均値 - A/C Set Temperature月別平均（A/C ON状態のみ）") 
                print("3. Indoortemp標準偏差 - Indoor Temp月別標準偏差（A/C ON状態のみ）")
                print("4. 設定温度_標準偏差 - A/C Set Temperature月別標準偏差（A/C ON状態のみ）")
                print("5. 室内機別_サンプル数 - A/C ON状態月別サンプル数")
                print("6. FanSpeed頻度 - A/C Fan Speed月別頻度")
            else:
                print(f"{building_name}: Excelファイル生成に失敗しました。")
        else:
            print(f"{building_name}: A/C Name別分析を実行できません。")
    else:
        print(f"{building_name}: A/C Name列がありません。")
        print(f"利用可能な列: {list(data.columns)}")

print(f"\n{'='*60}")
print("=== すべての建物分析完了 ===")
print(f"{'='*60}")



=== Clea 建物分析とExcelファイル生成 ===
総 26個のA/C Name発見

=== G-22 分析中 ===
  分析完了: 15個の月

=== G-21 分析中 ===
  分析完了: 15個の月

=== G-24 分析中 ===
  分析完了: 15個の月

=== G-23 分析中 ===
  分析完了: 7個の月

=== D-1南1 分析中 ===
  分析完了: 15個の月

=== D-3南2 分析中 ===
  分析完了: 15個の月

=== D-6北1 分析中 ===
  分析完了: 15個の月

=== D-5南1 分析中 ===
  分析完了: 15個の月

=== A-26 分析中 ===
  分析完了: 15個の月

=== E-17 分析中 ===
  分析完了: 15個の月

=== E-11南3 分析中 ===
  分析完了: 15個の月

=== E-9南1 分析中 ===
  分析完了: 15個の月

=== E-14北2 分析中 ===
  分析完了: 15個の月

=== E-13北1 分析中 ===
  分析完了: 15個の月

=== E-10南2 分析中 ===
  分析完了: 15個の月

=== F-18 分析中 ===
  分析完了: 15個の月

=== D-4北2 分析中 ===
  分析完了: 14個の月

=== E-15北3 分析中 ===
  分析完了: 15個の月

=== D-7南2 分析中 ===
  分析完了: 14個の月

=== D-8北2 分析中 ===
  分析完了: 13個の月

=== E-16北4 分析中 ===
  分析完了: 15個の月

=== A-25 分析中 ===
  分析完了: 15個の月

=== E-12南4 分析中 ===
  分析完了: 15個の月

=== D-2北1 分析中 ===
  分析完了: 12個の月

=== F-19 分析中 ===
  分析完了: 15個の月

=== F-20 分析中 ===
  分析完了: 15個の月
Clea: 総 26個のA/C Nameに対する分析完了
発見されたFan Speed値: ['Medium' 'Low' 'High' 'Top' 'Auto']


KeyboardInterrupt: 

# 室内温度-設定温度差分 シート追加


In [ ]:
# ========================================
# 新しいプロセス: 室内温度-設定温度差分シート追加
# ========================================

print("="*60)
print("新しいプロセス開始: 室内温度-設定温度差分シート追加")
print("="*60)

# 室内温度-設定温度差分シート生成関数定義
def create_temp_difference_dataframe(ac_analysis_results):
    """室内温度と設定温度の差分を計算してDataFrameを作成"""
    monthly_data = {}
    months = []
    
    # 月の順序を定義
    month_order = ['1月', '2月', '3月', '4月', '5月', '6月', 
                   '7月', '8月', '9月', '10月', '11月', '12月']
    
    # 各A/C Nameの月別データを処理
    for ac_name, monthly_stats in ac_analysis_results.items():
        monthly_data[ac_name] = {}
        
        for month in month_order:
            if month in monthly_stats['Indoor_Temp_Mean'] and month in monthly_stats['AC_Set_Temp_Mean']:
                # 室内温度平均 - 設定温度平均
                temp_diff = monthly_stats['Indoor_Temp_Mean'][month] - monthly_stats['AC_Set_Temp_Mean'][month]
                monthly_data[ac_name][month] = round(temp_diff, 2)
                
                if month not in months:
                    months.append(month)
    
    # DataFrame生成（月別が行、A/C Nameが列）
    df = pd.DataFrame(monthly_data, index=months)
    
    # 最初の列を月情報として追加
    df.insert(0, 'Unnamed: 0', months)
    
    return df

# 既存Excelファイルに室内温度-設定温度差分シート追加
def add_temp_difference_sheet_to_existing_file(building_name):
    """既存Excelファイルに室内温度-設定温度差分シート追加"""
    filename = f"AC_setvalue_range_analysis_{building_name}.xlsx"
    
    try:
        # 既存ファイル読み込み
        existing_excel = pd.ExcelFile(filename)
        existing_sheets = {}
        
        for sheet_name in existing_excel.sheet_names:
            existing_sheets[sheet_name] = pd.read_excel(filename, sheet_name=sheet_name)
        
        print(f"\n=== {building_name} 室内温度-設定温度差分シート追加 ===")
        
        # Indoortemp平均シートと設定温度_平均値シート読み込み
        if 'Indoortemp平均' in existing_sheets and '設定温度_平均値' in existing_sheets:
            indoor_avg_df = existing_sheets['Indoortemp平均']
            ac_set_avg_df = existing_sheets['設定温度_平均値']
            
            # 月別データ抽出
            months = indoor_avg_df['Unnamed: 0'].tolist()
            ac_names = indoor_avg_df.columns[1:].tolist()  # 最初のカラム(月)を除く
            
            # 差分計算
            temp_diff_data = {}
            temp_diff_data['Unnamed: 0'] = months
            
            for ac_name in ac_names:
                if ac_name in indoor_avg_df.columns and ac_name in ac_set_avg_df.columns:
                    # 該当A/C Nameの月別差分計算
                    indoor_values = indoor_avg_df[ac_name].values
                    ac_set_values = ac_set_avg_df[ac_name].values
                    
                    # 差分計算 (室内温度 - 設定温度)
                    diff_values = []
                    for i in range(len(indoor_values)):
                        if pd.notna(indoor_values[i]) and pd.notna(ac_set_values[i]):
                            diff_values.append(round(indoor_values[i] - ac_set_values[i], 2))
                        else:
                            diff_values.append(None)
                    
                    temp_diff_data[ac_name] = diff_values
            
            # DataFrame生成
            temp_diff_df = pd.DataFrame(temp_diff_data)
            
            # 新しいシート追加してファイル保存
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # 既存シート保存
                for sheet_name, sheet_data in existing_sheets.items():
                    sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
                
                # 新しいシート追加
                temp_diff_df.to_excel(writer, sheet_name='室内温度-設定温度差分', index=False)
            
            print(f"✅ {building_name}: 室内温度-設定温度差分シート追加完了")
            print(f"   - 月別室内温度平均と設定温度平均の差分計算")
            print(f"   - 正数: 室内温度が設定温度より高い")
            print(f"   - 負数: 室内温度が設定温度より低い")
            
            return True
        else:
            print(f"❌ {building_name}: 必要なシートが見つかりません。")
            return False
            
    except Exception as e:
        print(f"❌ {building_name}: シート追加中にエラー発生 - {e}")
        return False

print("室内温度-設定温度差分シート生成関数定義完了")

새로운 프로세스 시작: 室内温度-設定温度差分 시트 추가
室内温度-設定温度差分 시트 생성 함수 정의 완료


In [ ]:
# 既存Excelファイルに室内温度-設定温度差分シート追加実行
print("\n=== 既存Excelファイルに室内温度-設定温度差分シート追加実行 ===")

# CleaとIsetanMitsukoshiファイルにシート追加
buildings_to_update = ['Clea', 'IsetanMitsukoshi']

for building_name in buildings_to_update:
    print(f"\n--- {building_name} ファイル処理 ---")
    
    # ファイル存在確認
    filename = f"AC_setvalue_range_analysis_{building_name}.xlsx"
    try:
        # ファイルが存在するか確認
        import os
        if os.path.exists(filename):
            print(f"✅ {filename} ファイル発見")
            
            # シート追加実行
            success = add_temp_difference_sheet_to_existing_file(building_name)
            
            if success:
                print(f"✅ {building_name}: 室内温度-設定温度差分シート追加成功")
            else:
                print(f"❌ {building_name}: シート追加失敗")
        else:
            print(f"❌ {filename} ファイルが見つかりません。")
            print(f"   まずAC分析を実行してExcelファイルを生成してください。")
            
    except Exception as e:
        print(f"❌ {building_name} 処理中にエラー: {e}")

print("\n" + "="*60)
print("室内温度-設定温度差分シート追加プロセス完了")
print("="*60)


=== 기존 Excel 파일에 室内温度-設定温度差分 시트 추가 실행 ===

--- Clea 파일 처리 ---
✅ AC_setvalue_range_analysis_Clea.xlsx 파일 발견

=== Clea 室内温度-設定温度差分 시트 추가 ===
✅ Clea: 室内温度-設定温度差分 시트 추가 완료
   - 월별 실내온도 평균과 설정온도 평균의 차분 계산
   - 양수: 실내온도가 설정온도보다 높음
   - 음수: 실내온도가 설정온도보다 낮음
✅ Clea: 室内温度-設定温度差分 시트 추가 성공

--- IsetanMitsukoshi 파일 처리 ---
✅ AC_setvalue_range_analysis_IsetanMitsukoshi.xlsx 파일 발견

=== IsetanMitsukoshi 室内温度-設定温度差分 시트 추가 ===
✅ IsetanMitsukoshi: 室内温度-設定温度差分 시트 추가 완료
   - 월별 실내온도 평균과 설정온도 평균의 차분 계산
   - 양수: 실내온도가 설정온도보다 높음
   - 음수: 실내온도가 설정온도보다 낮음
✅ IsetanMitsukoshi: 室内温度-設定温度差分 시트 추가 성공

室内温度-設定温度差分 시트 추가 프로세스 완료


制御マスタへ反映

In [108]:
# ========================================
# 新しいプロセス: MASTER_Clea.xlsx 制御マスタシート修正
# ========================================

print("="*60)
print("新しいプロセス開始: MASTER_Clea.xlsx 制御マスタシート修正")
print("="*60)

# MASTER_Clea.xlsxのMASTERシート構造確認
import pandas as pd

print("\n=== MASTER_Clea.xlsx MASTERシート構造確認 ===")
try:
    master_file = "MASTER_Clea.xlsx"
    master_excel = pd.ExcelFile(master_file)
    
    print(f"全体シート一覧: {master_excel.sheet_names}")
    
    # MASTERシートがあるか確認
    if "MASTER" in master_excel.sheet_names:
        print(f"\n✅ MASTERシート発見!")
        
        # MASTERシート読み込み
        master_df = pd.read_excel(master_file, sheet_name="MASTER")
        
        print(f"\nMASTERシート構造:")
        print(f"  行数: {len(master_df)}")
        print(f"  列数: {len(master_df.columns)}")
        print(f"  列名: {list(master_df.columns)}")
        
        print(f"\nMASTERシートサンプルデータ:")
        print(master_df.head(10))
        
        # 制御区分関連列確認
        control_columns = [col for col in master_df.columns if '制御' in col or 'control' in col.lower()]
        print(f"\n制御関連列: {control_columns}")
        
        # 環境予測区分関連列確認
        env_columns = [col for col in master_df.columns if '環境' in col or '予測' in col or 'env' in col.lower()]
        print(f"環境予測関連列: {env_columns}")
        
        # 制御区分の固有値確認
        if '制御区分' in master_df.columns:
            unique_control_types = master_df['制御区分'].unique()
            print(f"\n制御区分固有値: {unique_control_types}")
            
            # 各制御区分別データ確認
            for control_type in unique_control_types:
                control_data = master_df[master_df['制御区分'] == control_type]
                print(f"\n制御区分 '{control_type}' データ:")
                print(f"  行数: {len(control_data)}")
                print(f"  サンプルデータ:")
                print(control_data.head(3))
        
    else:
        print(f"\n❌ MASTERシートが見つかりません。")
        print(f"使用可能なシート: {master_excel.sheet_names}")
        
except Exception as e:
    print(f"ファイル読み込みエラー: {e}")

print("\n" + "="*60)
print("MASTERシート構造確認完了")
print("="*60)

新しいプロセス開始: MASTER_Clea.xlsx 制御マスタシート修正

=== MASTER_Clea.xlsx MASTERシート構造確認 ===
全体シート一覧: ['MASTER', '制御マスタ_エリア1', '制御マスタ', '設備マスタ', 'パラメータ', 'original']

✅ MASTERシート発見!

MASTERシート構造:
  行数: 26
  列数: 19
  列名: ['制御区分', '始業時間', '就業時間', '目標室内温度', '設定温度下限', '設定温度上限', '風量候補', '電力予測区分', 'PM Mesh ID', 'PM Addr ID', '実績電力', '電力変数キー', '環境予測区分', '環境変数キー', '実績設定温度', '実績室内温度', '実績運転', '実績モード', '実績風量']

MASTERシートサンプルデータ:
     制御区分      始業時間      就業時間  目標室内温度  設定温度下限  設定温度上限             風量候補 電力予測区分  \
0  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-1   
1  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-2   
2  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-3   
3  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-4   
4  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-4   
5  Area 1  07:30:00  20:00:00    22.5      19      26  Low,Medium,High   49-6   
6  Area 1  07:30:00  20:

In [113]:
# 風量候補 設定: AC_setvalue_range_analysis_Clea.xlsxから月別最頻値検索
print("\n=== 風量候補 設定 ロジック ===")

# AC_setvalue_range_analysis_Clea.xlsxのFanSpeed頻度シート読み込み
try:
    ac_analysis_file = "AC_setvalue_range_analysis_Clea.xlsx"
    fan_speed_df = pd.read_excel(ac_analysis_file, sheet_name="FanSpeed頻度")
    
    print(f"FanSpeed頻度シート構造:")
    print(f"  行数: {len(fan_speed_df)}")
    print(f"  列数: {len(fan_speed_df.columns)}")
    print(f"  列名: {list(fan_speed_df.columns)}")
    
    print(f"\nFanSpeed頻度サンプルデータ:")
    print(fan_speed_df.head(10))
    
    # 月別最頻値検索関数
    def find_monthly_mode_fan_speed(fan_speed_df):
        """各月別に最頻値Fan Speed検索"""
        monthly_mode = {}
        
        # 月別にグループ化
        months = fan_speed_df['Unnamed: 0'].unique()
        
        for month in months:
            month_data = fan_speed_df[fan_speed_df['Unnamed: 0'] == month]
            
            # 各Fan Speed別総頻度計算
            fan_speed_totals = {}
            fan_speeds = ['Medium', 'Low', 'High', 'Top', 'Auto']
            
            for fan_speed in fan_speeds:
                if fan_speed in month_data['Unnamed: 1'].values:
                    # 該当Fan Speedの総頻度計算 (全A/C Name合計)
                    fan_speed_rows = month_data[month_data['Unnamed: 1'] == fan_speed]
                    total_frequency = fan_speed_rows['frequency'].sum()
                    fan_speed_totals[fan_speed] = total_frequency
            
            # 最頻値検索
            if fan_speed_totals:
                mode_fan_speed = max(fan_speed_totals, key=fan_speed_totals.get)
                monthly_mode[month] = {
                    'mode': mode_fan_speed,
                    'frequency': fan_speed_totals[mode_fan_speed],
                    'all_frequencies': fan_speed_totals
                }
            else:
                monthly_mode[month] = {
                    'mode': 'Medium',  # デフォルト値
                    'frequency': 0,
                    'all_frequencies': {}
                }
        
        return monthly_mode
    
    # 月別最頻値計算
    monthly_mode_results = find_monthly_mode_fan_speed(fan_speed_df)
    
    print(f"\n=== 月別最頻値Fan Speed結果 ===")
    for month, result in monthly_mode_results.items():
        print(f"{month}: {result['mode']} (頻度: {result['frequency']})")
        print(f"  全体頻度: {result['all_frequencies']}")
    
    # 結果をDataFrameで整理
    mode_summary = []
    for month, result in monthly_mode_results.items():
        mode_summary.append({
            '月': month,
            '風量候補': result['mode'],
            '頻度': result['frequency']
        })
    
    mode_df = pd.DataFrame(mode_summary)
    print(f"\n=== 風量候補要約 ===")
    print(mode_df)
    
except Exception as e:
    print(f"FanSpeed頻度シート読み込みエラー: {e}")

print("\n" + "="*60)
print("風量候補設定ロジック完了")
print("="*60)


=== 風量候補 設定 ロジック ===
FanSpeed頻度シート構造:
  行数: 60
  列数: 29
  列名: ['Unnamed: 0', 'Unnamed: 1', 'frequency', 'G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

FanSpeed頻度サンプルデータ:
  Unnamed: 0 Unnamed: 1  frequency  G-22  G-21  G-24  G-23  D-1南1  D-3南2  \
0         1月     Medium      19630  1286  1626    50     0   6642   6642   
1         1月        Low      35902     0     0     0     0   2206   2206   
2         1月       High        256     0     0     0     0      0      0   
3         1月        Top          0     0     0     0     0      0      0   
4         1月       Auto          0     0     0     0     0      0      0   
5         2月     Medium      22292  1369  1583   198     0   7816   7750   
6         2月        Low      29167     0     0     0     0      0      0   
7         2月       High        148  

In [114]:
# ========================================
# 制御マスタシート温度カラムアップデートロジック
# ========================================

print("="*60)
print("制御マスタシート温度カラムアップデートロジック")
print("="*60)

# AC_setvalue_range_analysis_Clea.xlsxから温度データ読み込み
def load_temperature_data():
    """AC_setvalue_range_analysis_Clea.xlsxから温度関連シート読み込み"""
    try:
        ac_analysis_file = "AC_setvalue_range_analysis_Clea.xlsx"
        ac_excel = pd.ExcelFile(ac_analysis_file)
        
        print(f"AC分析ファイルシート一覧: {ac_excel.sheet_names}")
        
        # 必要なシート読み込み
        temp_sheets = {}
        
        if '設定温度_平均値' in ac_excel.sheet_names:
            temp_sheets['avg'] = pd.read_excel(ac_analysis_file, sheet_name='設定温度_平均値')
            print("✅ 設定温度_平均値シート読み込み完了")
        else:
            print("❌ 設定温度_平均値シートが見つかりません。")
            
        if '設定温度_標準偏差' in ac_excel.sheet_names:
            temp_sheets['std'] = pd.read_excel(ac_analysis_file, sheet_name='設定温度_標準偏差')
            print("✅ 設定温度_標準偏差シート読み込み完了")
        else:
            print("❌ 設定温度_標準偏差シートが見つかりません。")
            
        return temp_sheets
        
    except Exception as e:
        print(f"温度データ読み込みエラー: {e}")
        return None

# 制御区分別温度データマッピング関数
def map_temperature_data_by_control_type(temp_sheets, control_env_mapping):
    """各制御区分別に対応するA/C Nameの温度データをマッピング"""
    if not temp_sheets or 'avg' not in temp_sheets or 'std' not in temp_sheets:
        print("❌ 温度データがありません。")
        return None
    
    avg_df = temp_sheets['avg']
    std_df = temp_sheets['std']
    
    # 月別データ抽出
    months = avg_df['Unnamed: 0'].tolist()
    ac_names = avg_df.columns[1:].tolist()  # 最初のカラム(月)を除く
    
    print(f"月別データ: {months}")
    print(f"A/C Name一覧: {ac_names}")
    
    # 各制御区分別温度データマッピング
    control_temp_data = {}
    
    for control_type, env_list in control_env_mapping.items():
        if control_type == 'nan':
            continue
            
        print(f"\n--- {control_type} 温度データマッピング ---")
        print(f"該当環境予測区分: {env_list}")
        
        # 該当制御区分に属するA/C Name検索
        matching_ac_names = []
        for env in env_list:
            if env in ac_names:
                matching_ac_names.append(env)
        
        print(f"マッチングA/C Name: {matching_ac_names}")
        
        if matching_ac_names:
            # 該当A/C Nameの月別平均温度計算
            monthly_avg_temps = {}
            monthly_std_temps = {}
            
            for month in months:
                month_idx = months.index(month)
                
                # 該当月の全マッチングA/C Nameの平均温度計算
                avg_values = []
                std_values = []
                
                for ac_name in matching_ac_names:
                    if ac_name in avg_df.columns and ac_name in std_df.columns:
                        avg_val = avg_df.iloc[month_idx][ac_name]
                        std_val = std_df.iloc[month_idx][ac_name]
                        
                        if pd.notna(avg_val) and pd.notna(std_val):
                            avg_values.append(avg_val)
                            std_values.append(std_val)
                
                if avg_values and std_values:
                    # 平均温度の平均
                    monthly_avg_temps[month] = round(sum(avg_values) / len(avg_values), 2)
                    # 標準偏差の平均
                    monthly_std_temps[month] = round(sum(std_values) / len(std_values), 2)
                else:
                    monthly_avg_temps[month] = None
                    monthly_std_temps[month] = None
            
            control_temp_data[control_type] = {
                'avg_temps': monthly_avg_temps,
                'std_temps': monthly_std_temps,
                'ac_names': matching_ac_names
            }
            
            print(f"  月別平均温度: {monthly_avg_temps}")
            print(f"  月別標準偏差: {monthly_std_temps}")
        else:
            print(f"  ❌ {control_type}にマッチングするA/C Nameがありません。")
    
    return control_temp_data

# 温度データロード及びマッピング実行
print("\n=== 温度データロード及びマッピング ===")
temp_sheets = load_temperature_data()

if temp_sheets:
    # MASTERシートから制御区分別環境予測区分マッピング取得
    master_file = "MASTER_Clea.xlsx"
    master_df = pd.read_excel(master_file, sheet_name="MASTER")
    
    control_env_mapping = {}
    for _, row in master_df.iterrows():
        control_type = row['制御区分']
        env_prediction = row['環境予測区分']
        
        if pd.notna(control_type) and pd.notna(env_prediction):
            if control_type not in control_env_mapping:
                control_env_mapping[control_type] = []
            control_env_mapping[control_type].append(env_prediction)
    
    print(f"\n制御区分別環境予測区分マッピング: {control_env_mapping}")
    
    # 温度データマッピング
    control_temp_data = map_temperature_data_by_control_type(temp_sheets, control_env_mapping)
    
    if control_temp_data:
        print(f"\n✅ 温度データマッピング完了: {len(control_temp_data)}個制御区分")
    else:
        print("❌ 温度データマッピング失敗")
else:
    print("❌ 温度データロード失敗")

print("\n" + "="*60)
print("温度データロード及びマッピング完了")
print("="*60)

制御マスタシート温度カラムアップデートロジック

=== 温度データロード及びマッピング ===
AC分析ファイルシート一覧: ['Indoortemp平均', '設定温度_平均値', 'Indoortemp標準偏差', '設定温度_標準偏差', '室内機別_サンプル数', 'FanSpeed頻度', '室内温度-設定温度差分']
✅ 設定温度_平均値シート読み込み完了
✅ 設定温度_標準偏差シート読み込み完了

制御区分別環境予測区分マッピング: {'Area 1': ['E-16北4', 'E-15北3', 'E-12南4', 'E-11南3', 'E-14北2', 'E-13北1', 'E-10南2', 'E-9南1'], 'Area 2': ['D-8北2', 'D-6北1', 'D-7南2', 'D-5南1', 'D-4北2', 'D-2北1'], 'Area 3': ['E-17', 'A-26'], 'Area 4': ['F-18', 'F-20', 'F-19'], 'Meeting Room': ['G-24', 'G-23'], 'Break Room': ['G-21', 'G-22']}
月別データ: ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
A/C Name一覧: ['G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

--- Area 1 温度データマッピング ---
該当環境予測区分: ['E-16北4', 'E-15北3', 'E-12南4', 'E-11南3', 'E-14北2', 'E-13北1', 'E-10南2', 'E-9南1']
マッチングA/C Name: ['E-16北4', 'E-15北3', 'E-12

In [115]:
# ========================================
# 制御マスタシート統合アップデート (新しい方針)
# ========================================

print("\n=== 制御マスタシート統合アップデート (新しい方針) ===")

if 'control_temp_data' in locals() and control_temp_data:
    try:
        # 既存MASTER_Clea.xlsxファイル読み込み
        master_file = "MASTER_Clea.xlsx"
        master_excel = pd.ExcelFile(master_file)
        
        # 既存シート読み込み
        existing_sheets = {}
        for sheet_name in master_excel.sheet_names:
            existing_sheets[sheet_name] = pd.read_excel(master_file, sheet_name=sheet_name)
        
        print(f"既存シート一覧: {list(existing_sheets.keys())}")
        
        # 月別風量候補データ取得 (セル20で計算された結果使用)
        if 'monthly_mode_results' in locals():
            monthly_fan_speed = {}
            for month, result in monthly_mode_results.items():
                monthly_fan_speed[month] = result['mode']
            print(f"月別風量候補: {monthly_fan_speed}")
        else:
            print("monthly_mode_resultsが定義されていません。デフォルト値使用")
            monthly_fan_speed = {
                '1月': 'Low', '2月': 'Low', '3月': 'Low', '4月': 'Low',
                '5月': 'Low', '6月': 'Low', '7月': 'Low', '8月': 'Low',
                '9月': 'Low', '10月': 'Low', '11月': 'Low', '12月': 'Low'
            }
        
        # 必要なデータロード
        print(f"\n=== 必要なデータロード ===")
        try:
            ac_analysis_file = "AC_setvalue_range_analysis_Clea.xlsx"
            
            # Indoor Temp.データロード
            indoor_avg_df = pd.read_excel(ac_analysis_file, sheet_name='Indoortemp平均')
            indoor_std_df = pd.read_excel(ac_analysis_file, sheet_name='Indoortemp標準偏差')
            print(f"✅ Indoor Temp.平均シート読み込み完了: {indoor_avg_df.shape}")
            print(f"✅ Indoor Temp.標準偏差シート読み込み完了: {indoor_std_df.shape}")
            
            # 設定温度データロード
            set_temp_avg_df = pd.read_excel(ac_analysis_file, sheet_name='設定温度_平均値')
            set_temp_std_df = pd.read_excel(ac_analysis_file, sheet_name='設定温度_標準偏差')
            print(f"✅ 設定温度平均シート読み込み完了: {set_temp_avg_df.shape}")
            print(f"✅ 設定温度標準偏差シート読み込み完了: {set_temp_std_df.shape}")
            
            # 月別データ抽出
            months = indoor_avg_df['Unnamed: 0'].tolist()
            ac_names = indoor_avg_df.columns[1:].tolist()  # 最初のカラム(月)を除く
            
            print(f"月別データ: {months}")
            print(f"A/C Name一覧: {ac_names}")
            
        except Exception as e:
            print(f"❌ データロード失敗: {e}")
            indoor_avg_df = None
            indoor_std_df = None
            set_temp_avg_df = None
            set_temp_std_df = None
        
        # 新しい統合データフレーム生成
        print(f"\n=== 統合データ生成 ===")
        
        # 各制御区画別にデータ生成
        all_rows = []
        
        for control_type, temp_data in control_temp_data.items():
            print(f"\n--- {control_type} データ処理 ---")
            print(f"  マッチングA/C Name: {temp_data['ac_names']}")
            
            # 各月別にデータ計算
            for month in months:
                if month in monthly_fan_speed:
                    month_df_idx = months.index(month)
                    
                    # マッチングするA/C Nameのデータ計算
                    indoor_avg_values = []
                    indoor_std_values = []
                    set_temp_avg_values = []
                    set_temp_std_values = []
                    
                    for ac_name in temp_data['ac_names']:
                        # Indoor Temp.データ
                        if ac_name in indoor_avg_df.columns and ac_name in indoor_std_df.columns:
                            indoor_avg_val = indoor_avg_df.iloc[month_df_idx][ac_name]
                            indoor_std_val = indoor_std_df.iloc[month_df_idx][ac_name]
                            
                            if pd.notna(indoor_avg_val) and pd.notna(indoor_std_val):
                                indoor_avg_values.append(indoor_avg_val)
                                indoor_std_values.append(indoor_std_val)
                        
                        # 設定温度データ
                        if ac_name in set_temp_avg_df.columns and ac_name in set_temp_std_df.columns:
                            set_temp_avg_val = set_temp_avg_df.iloc[month_df_idx][ac_name]
                            set_temp_std_val = set_temp_std_df.iloc[month_df_idx][ac_name]
                            
                            if pd.notna(set_temp_avg_val) and pd.notna(set_temp_std_val):
                                set_temp_avg_values.append(set_temp_avg_val)
                                set_temp_std_values.append(set_temp_std_val)
                    
                    if indoor_avg_values and indoor_std_values and set_temp_avg_values and set_temp_std_values:
                        # 平均計算
                        avg_indoor_temp = sum(indoor_avg_values) / len(indoor_avg_values)
                        avg_indoor_std = sum(indoor_std_values) / len(indoor_std_values)
                        avg_set_temp = sum(set_temp_avg_values) / len(set_temp_avg_values)
                        avg_set_temp_std = sum(set_temp_std_values) / len(set_temp_std_values)
                        
                        # 温度値計算 (既存ロジックをそのまま使用)
                        # 目標室内温度上限, 目標室内温度下限は0.1単位で丸め
                        target_indoor_upper = round((avg_indoor_temp + avg_indoor_std) * 10) / 10
                        target_indoor_lower = round((avg_indoor_temp - avg_indoor_std) * 10) / 10
                        
                        # 設定温度上限, 設定温度下限は0.5単位で丸め
                        set_temp_upper = round((avg_set_temp + avg_set_temp_std) * 2) / 2
                        set_temp_lower = round((avg_set_temp - avg_set_temp_std) * 2) / 2
                        
                        # 風量候補
                        fan_speed = monthly_fan_speed[month]
                        
                        # 新しい行生成
                        new_row = {
                            '月': month,
                            '制御区画': control_type,
                            '目標室内温度上限': target_indoor_upper,
                            '目標室内温度下限': target_indoor_lower,
                            '設定温度上限': set_temp_upper,
                            '設定温度下限': set_temp_lower,
                            '風量候補': fan_speed
                        }
                        
                        all_rows.append(new_row)
                        
                        print(f"    {month}: 目標室内温度上限={target_indoor_upper}°C, 目標室内温度下限={target_indoor_lower}°C, 設定温度上限={set_temp_upper}°C, 設定温度下限={set_temp_lower}°C, 風量={fan_speed}")
                    else:
                        print(f"    {month}: データがありません。")
        
        # 新しい統合データフレーム生成
        if all_rows:
            new_integrated_df = pd.DataFrame(all_rows)
            print(f"\n✅ 統合データフレーム生成完了: {new_integrated_df.shape}")
            print(f"新しいカラム: {new_integrated_df.columns.tolist()}")
            
            # 既存シート更新
            updated_sheets = existing_sheets.copy()
            updated_sheets['制御マスタ'] = new_integrated_df
            
            # アップデートされたファイル保存
            print(f"\n=== アップデートされたファイル保存 ===")
            output_file = "MASTER_Clea_integrated.xlsx"
            
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                for sheet_name, df in updated_sheets.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"  {sheet_name} シート保存完了")
            
            print(f"\n✅ 統合された制御マスタシートファイル保存完了: {output_file}")
            
            # 結果確認
            print(f"\n=== 結果確認 ===")
            updated_excel = pd.ExcelFile(output_file)
            print(f"新しいファイルシート一覧: {updated_excel.sheet_names}")
            
            # 統合された制御マスタシート確認
            if '制御マスタ' in updated_excel.sheet_names:
                integrated_sheet = pd.read_excel(output_file, sheet_name='制御マスタ')
                print(f"\n統合された制御マスタシート確認:")
                print(f"シートサイズ: {integrated_sheet.shape}")
                print(f"カラム: {integrated_sheet.columns.tolist()}")
                print(f"\n最初の10行:")
                print(integrated_sheet.head(10))
                
                # 制御区画別データ数確認
                print(f"\n制御区画別データ数:")
                print(integrated_sheet['制御区画'].value_counts())
                
                # 月別データ数確認
                print(f"\n月別データ数:")
                print(integrated_sheet['月'].value_counts().sort_index())
        else:
            print("❌ 生成するデータがありません。")
        
    except Exception as e:
        print(f"❌ シートアップデート中にエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ 温度データがありません。まずセル21を実行してください。")

print("\n" + "="*60)
print("制御マスタシート統合アップデート完了")
print("="*60)


=== 制御マスタシート統合アップデート (新しい方針) ===
既存シート一覧: ['MASTER', '制御マスタ_エリア1', '制御マスタ', '設備マスタ', 'パラメータ', 'original']
月別風量候補: {'1月': 'Low', '2月': 'Low', '3月': 'Low', '4月': 'Low', '5月': 'Low', '6月': 'Low', '7月': 'Low', '8月': 'Low', '9月': 'Low', '10月': 'Low', '11月': 'Low', '12月': 'Low'}

=== 必要なデータロード ===
✅ Indoor Temp.平均シート読み込み完了: (12, 27)
✅ Indoor Temp.標準偏差シート読み込み完了: (12, 27)
✅ 設定温度平均シート読み込み完了: (12, 27)
✅ 設定温度標準偏差シート読み込み完了: (12, 27)
月別データ: ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
A/C Name一覧: ['G-22', 'G-21', 'G-24', 'G-23', 'D-1南1', 'D-3南2', 'D-6北1', 'D-5南1', 'A-26', 'E-17', 'E-11南3', 'E-9南1', 'E-14北2', 'E-13北1', 'E-10南2', 'F-18', 'D-4北2', 'E-15北3', 'D-7南2', 'D-8北2', 'E-16北4', 'A-25', 'E-12南4', 'D-2北1', 'F-19', 'F-20']

=== 統合データ生成 ===

--- Area 1 データ処理 ---
  マッチングA/C Name: ['E-16北4', 'E-15北3', 'E-12南4', 'E-11南3', 'E-14北2', 'E-13北1', 'E-10南2', 'E-9南1']
    1月: 目標室内温度上限=27.3°C, 目標室内温度下限=22.3°C, 設定温度上限=26.0°C, 設定温度下限=22.5°C, 風量=Low
    2月: 目標室内温度上限=27.2°C, 目標室内温度下